In [1]:
%reload_ext autoreload
%autoreload
%matplotlib inline
import os, json,glob,sys,io
from types import SimpleNamespace
from experiment import run_model
from eval import calculate_stats, eval_model
import pickle
from datetime import datetime 
import torch
from transformers import BertTokenizer
from model.MedClinical import Biobert_fc 
from sklearn.metrics import classification_report, f1_score
from util.tools  import load_config
from util.DataLoader  import *
from pathlib import Path, PureWindowsPath, PurePosixPath
import traceback
from matplotlib import pyplot as plt
import pandas as pd

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"

In [3]:
config_folder = "config/"
config = load_config(config_folder)

In [4]:
dataprocessor = MultiClassificationProcessor()
test_dataloader, dev_data_len, dev_num_labels, dev_num_train_optimization_steps, all_dev_label_ids = dataprocessor.get_data_loader(config,source='test')    

INFO:root:***** Running training *****
INFO:root:  Num examples = 1000
INFO:root:  Batch size = 24
INFO:root:  Num steps = 410


In [ ]:
models_folder_name = config.programsettings.OUTPUT_DIR
data_folder = Path(models_folder_name)
list_model_files = list(data_folder.glob('**/*.bin'))
num_labels = 9

f1_score_list = []
for model_file in list_model_files:
    try:
        print("\n Model file: ", model_file)
        model = torch.load(model_file)
        x =model.eval()
        test_inputs, test_preds, test_labels, test_loss = eval_model( config, model, test_dataloader, device, num_labels)  
        print(classification_report(test_labels,test_preds ))
        print("\n \n \n")
        f1_value = f1_score(test_labels, test_preds,average='macro')
        f1_score_list.append([model_file,f1_value ])
        
    except:
        print("Failed due to cpu & GPU incompatibility")
     # printing stack trace 
#         traceback.print_exc()         

print("\n \n \n")





 Model file:  outputs\re\BERT_Sequence2020-11-15 15_34_27_348184.bin



Failed due to cpu & GPU incompatibility

 Model file:  outputs\re\BERT_Sequence2020-11-15 18_33_16_910419.bin



Failed due to cpu & GPU incompatibility

 Model file:  outputs\re\BERT_Sequence2020-11-16 05_43_28_786014.bin



              precision    recall  f1-score   support

           0       0.76      1.00      0.86        44
           1       0.68      0.96      0.80        53
           2       0.94      0.98      0.96        50
           3       1.00      0.96      0.98        55
           4       0.90      0.90      0.90        10
           5       0.97      0.58      0.73        57
           6       0.98      0.93      0.96        46
           7       1.00      0.67      0.80        33
           8       1.00      1.00      1.00       652

    accuracy                           0.96      1000
   macro avg       0.91      0.89      0.89      1000
weighted avg       0.97      0.96      0.95      1000


 
 


 Model file:  outputs\re\BERT_Sequence2020-11-16 10_27_50_320534.bin



Failed due to cpu & GPU incompatibility

 Model file:  outputs\re\BERT_Sequence2020-11-16 22_21_41_536138.bin


...


              precision    recall  f1-score   support

           0       0.65      0.98      0.78        44
           1       0.84      0.96      0.89        53
           2       0.98      0.96      0.97        50
           3       0.95      0.96      0.95        55
           4       0.82      0.90      0.86        10
           5       1.00      0.72      0.84        57
           6       0.95      0.87      0.91        46
           7       0.89      0.73      0.80        33
           8       1.00      0.99      1.00       652

    accuracy                           0.96      1000
   macro avg       0.90      0.90      0.89      1000
weighted avg       0.96      0.96      0.96      1000


 
 


 Model file:  outputs\re\BioBERT_fc2020-11-16 03_37_07_579463.bin



              precision    recall  f1-score   support

           0       0.86      1.00      0.93        44
           1       0.82      1.00      0.90        53
           2       0.96      0.96      0.96        50
           3       0.98      0.98      0.98        55
           4       0.90      0.90      0.90        10
           5       1.00      0.63      0.77        57
           6       0.96      0.96      0.96        46
           7       0.81      0.88      0.84        33
           8       1.00      1.00      1.00       652

    accuracy                           0.97      1000
   macro avg       0.92      0.92      0.92      1000
weighted avg       0.97      0.97      0.97      1000


 
 


 Model file:  outputs\re\BioBERT_fc2020-11-16 09_42_50_419072.bin



Failed due to cpu & GPU incompatibility

 Model file:  outputs\re\BioBERT_fc2020-11-16 09_54_01_216248.bin



Failed due to cpu & GPU incompatibility

 Model file:  outputs\re\BioBERT_fc2020-11-16 10_17_23_898361.bin



Failed due to cpu & GPU incompatibility

 Model file:  outputs\re\BioBERT_fc2020-11-16 16_17_55_396529.bin


In [ ]:
pd.set_option('display.max_colwidth', 120)
columns = ['file_namae', 'f1-score']
test_results= pd.DataFrame(f1_score_list, columns = columns)
test_results

In [ ]:
# print ("F1 Score summary \n")  
# ax = plt.gca()
# ax.get_xaxis().set_visible(False)
# ax.get_yaxis().set_visible(False)
# plt.box(on=None)
# plt.figure(figsize=(14,6))
# the_table = plt.table(cellText=f1_score_list,
#                       colLabels=['file name', 'F1-score'],
#                       loc='center')
# the_table.auto_set_font_size(False)
# the_table.set_fontsize(10)